In [5]:
import pandas as pd 
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
df.drop("RID",axis=1, inplace = True) #RID는 그냥 Index라서 삭제

In [6]:
df

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## 함수 만들기

In [7]:
import math

def getEntropy(df, feature) :
    
    at_class = df[feature].value_counts() #전달받은 데이터프레임의 index, value를 담음. ex) Class_buys_computer의 경우 y/n과 그에 대한 각각의 value 저장
    
    if len(at_class) == 1: #y혹은 n하나만 있는 경우 entropy는 0이기 때문에 미리 처리해줌.
        entropy = 0
    
    else: #두 값이 다 존재하는 경우
        yes = at_class['yes']/len(df) #yes인 경우
        no = at_class['no']/len(df) #no인 경우

        entropy =  -yes * math.log(yes,2) -no * math.log(no,2) #엔트로피를 계산
   
    return entropy

In [8]:
getEntropy(df, 'class_buys_computer')

0.9402859586706309

In [10]:
def getGainA(df, feature) :
    info_D = getEntropy(df, feature) # 목표변수 Feature에 대한 Info(Entropy)를 구한다.
    columns = list(df.loc[:, df.columns != feature]) # 목표변수를 제외한 나머지 설명변수들을 리스트 형태로 저장한다.
    
    gains = [] #각 columns에 대한 최종 엔트로피 값을 저장할 리스트 선언
    for i in range(len(columns)):
        gains_c = [] #각 columns의 class에 대한 엔트로피값을 저장할 리스트, 이는 columns이 시작할 때마다 초기화된다.
        filt_class = df[columns[i]].value_counts().index # columns에 대한 클래스 정보를 저장. ex) 'age'의 경우 filt_class에는 'youth', 'middle_aged', 'senior'
        
        for j in range(len(filt_class)): #그 columns에 대한 class를 순회 ex) 'age' columns의 경우 차례대로 youth, middle_aged, senior을 순회함.
            filt_data = df[df[columns[i]].isin([filt_class[j]])] #해당 columns의 class만을 필터링한 데이터            
            get_entro = getEntropy(filt_data, feature) #filter된 데이터 셋에서 class_buys_computer에 대한 엔트로피를 구함.
            weight = len(filt_data)/len(df) #가중치를 계산함.
            Entropy = weight * get_entro #최종 엔트로피 계산
            gains_c.append(Entropy)#이를 gains_c에 저장함. 그리고 총 이 작업이 class의 개수만큼 일어난다.
        
        gains.append(sum(gains_c)) #columns 하나를 다 돌았다면 그 sum 값을 gains에 담는다.
    
    gains = info_D - gains #이걸 아까 위에서 구한 총 엔트로피에서 빼줌.
    result = dict(zip(columns,gains)) # 각 변수에 대한 Information Gain 을 Dictionary 형태로 저장
    return(result)#출력~

In [11]:
getGainA(df, "class_buys_computer")

{'age': 0.2467498197744391,
 'income': 0.029222565658954647,
 'student': 0.15183550136234136,
 'credit_rating': 0.04812703040826927}

## 결과 확인하기

In [12]:
my_dict = getGainA(df, "class_buys_computer")
def f1(x):
    return my_dict[x]
key_max = max(my_dict.keys(), key=f1)
print('정보 획득이 가장 높은 변수는',key_max, "이며 정보 획득량은", my_dict[key_max], "이다.")

정보 획득이 가장 높은 변수는 age 이며 정보 획득량은 0.2467498197744391 이다.


In [13]:
# V ^------------^ V 